In [1]:
import sys; sys.path.insert(0,'/Users/ryan/github/prosodic/')
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.imports import *

In [144]:
#txt='apple banana, celery! donut eagle, fire george henry igloo apple banana celery donut eagle fire george henry igloo apple banana celery donut eagle fire george henry igloo apple banana celery donut eagle fire george henry igloo'
txt='the '*100
txt
# txt = sonnet

'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the '

In [145]:
t = Text(txt)
doc = t.para(1)
df=doc.sylls(index=False)
# df

In [142]:
def divide_parse_units(dfwords,nsyll=None,linebreaks=LINEBREAKS,phrasebreaks=PHRASEBREAKS):
    # reverse
    if linebreaks and not phrasebreaks:
        dfwords['unit_i']=divide_by_line_only(dfwords)
    elif phrasebreaks and not linebreaks:
        dfwords['unit_i']=divide_by_phrase_only(dfwords)
    elif phrasebreaks and linebreaks:
        dfwords['unit_i']=divide_by_phrase_and_line(dfwords)
    
    dfwords['unit_i']=list(divide_overflow(dfwords))
    dfwords['unit_i']=dfwords['unit_i'].rank(method='dense').apply(int)


def divide_by_line_only(dfwords): return divide_by(dfwords,by=['line_i'])
def divide_by_phrase_only(dfwords): return divide_by(dfwords,by=['sent_i','sentpart_i'])
def divide_by_phrase_and_line(dfwords): return divide_by(dfwords,by=['sent_i','sentpart_i','line_i'])
def divide_by(dfwords, by=['sent_i','sentpart_i','line_i']):
    o=[]
    keynow=None
    unit_i=0
    inp = (dfwords[k] for k in by)
    for rowkey in zip(*inp):
        if rowkey!=keynow: unit_i+=1
        o.append(unit_i)
        keynow=rowkey
    return o

        
MAX_SYLL_IN_PARSE_UNIT=14
MIN_SYLL_IN_PARSE_UNIT=None

def get_nsyll(df):
    return len(df[df.word_ipa_i==1])

def divide_overflow(dfsyll,max_nsyll=MAX_SYLL_IN_PARSE_UNIT):
    for ui,unitdf in dfsyll.groupby('unit_i'):
        nsyll=get_nsyll(unitdf)
        if nsyll<=max_nsyll:
            for n in range(len(unitdf)):
                yield ui
        else:
            newui=[]
            nsyllnow=0
            ui2=2
            gl=list(unitdf.groupby(['sent_i','word_i']))
            for wi,wdf in reversed(gl):
                nsyll_word=get_nsyll(wdf)
                for si in range(len(wdf)): newui+=[ui + (1/ui2)]
                nsyllnow+=nsyll_word
                if nsyllnow>=max_nsyll:
                    ui2+=1
                    nsyllnow=0
            yield from reversed(newui)

In [143]:
divide_parse_units(df)
# df.head(25)
df

,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok,word_ipa_i,syll_i,word_ipa,word_nsyll,syll_ipa,syll_str,syll_stress,prom_stress,word_isfunc,prom_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,unit_i
0,1,1,1,1,,How,how,1,1,haʊ,1,haʊ,how,U,0.0,1,NaN,NaN,0,1,0,0,0,0,1
1,1,1,1,2,,heavy,heavy,1,1,'hɛ.viː,2,'hɛ,hea,P,1.0,0,NaN,1.0,1,0,0,0,1,0,1
2,1,1,1,2,,heavy,heavy,1,2,'hɛ.viː,2,viː,vy,U,0.0,0,NaN,0.0,0,1,0,0,0,1,1
3,1,1,1,3,,do,do,1,1,'duː,1,'duː,do,P,1.0,0,NaN,NaN,1,0,0,0,0,0,1
4,1,1,1,3,,do,do,2,1,duː,1,duː,do,U,0.0,1,NaN,NaN,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,2,15,14,81,,and,and,1,1,ænd,1,ænd,and,U,0.0,1,NaN,NaN,0,1,0,0,0,0,18
166,2,15,14,82,,my,my,1,1,maɪ,1,maɪ,my,U,0.0,1,NaN,NaN,0,1,0,0,0,0,18
167,2,15,14,83,,joy,joy,1,1,'ʤɔɪ,1,'ʤɔɪ,joy,P,1.0,0,NaN,NaN,1,0,0,0,0,0,18
168,2,15,14,84,,behind.,behind,1,1,bɪ.'haɪnd,2,bɪ,be,U,0.0,0,NaN,0.0,0,1,0,0,0,1,18


In [104]:
list(divide_overflow(df))

[(1, 3),
 (1, 3),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1)]

In [34]:
print(sonnet.strip().count('\n'))

13


In [29]:
divide_parse_units(dfwords)

117